# User interface

## Command `sos`

Command `sos` accepts a number of subcommands (similar to `svn`, `git` etc). Its syntax follows

```bash
sos subcommand [subcommand-options]
```

You can use command

```bash
$ sos -h
usage: sos [-h] [--version]
           {run,dryrun,prepare,convert,remove,start,config} ...

A workflow system for the execution of commands and scripts in different
languages.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit

subcommands:
  {run,dryrun,prepare,convert,remove,start,config}
    run                 Execute a SoS script
    dryrun              Execute a SoS script in dryrun mode
    prepare             Execute a SoS script in prepare mode
    convert             Convert between sos and other file formats such as
                        html and Jupyter notebooks
    remove              Remove tracked and/or untracked files with their
                        signatures
    config              Set, unset or get the value of system or local
                        configuration files

Use 'sos cmd -h' for details about each subcommand. Please contact Bo Peng
(bpeng at mdanderson.org) if you have any question.
```

to get a list of subcommands with brief descriptions and

```bash
sos subcommand -h
```

to get detailed description of a particular subcommand.

## subcommand `run`

```
$ sos run -h
usage: sos run [-h] [-j JOBS] [-c CONFIG_FILE] [-t FILE [FILE ...]]
               [-b [BIN_DIR [BIN_DIR ...]]] [-q QUEUE] [-n] [-p] [-f] [-F]
               [-v {0,1,2,3,4}]
               SCRIPT [WORKFLOW]

Execute a workflow defined in script

positional arguments:
  SCRIPT                A SoS script that defines one or more workflows. The
                        script can be a filename or a URL from which the
                        content of a SoS will be read. If a valid file cannot
                        be located or downloaded, SoS will search for the
                        script in a search path specified by variable
                        `sos_path` defined in the global SoS configuration
                        file (~/.sos/config.yaml).
  WORKFLOW              Name of the workflow to execute. This option can be
                        ignored if the script defines a default workflow (with
                        no name or with name `default`) or defines only a
                        single workflow. A subworkflow or a combined workflow
                        can also be specified, where a subworkflow executes a
                        subset of workflow (`name_steps` where `steps` can be
                        `n` (a step `n`), `:n` (up to step `n`), `n:m` (from
                        step `n` to `m`), and `n:` (from step `n`)), and a
                        combined workflow executes to multiple (sub)workflows
                        combined by `+` (e.g. `A_0+B+C`).

optional arguments:
  -h, --help            show this help message and exit
  -j JOBS               Number of concurrent process allowed. A workflow is by
                        default executed sequentially (-j 1). If a greater
                        than 1 number is specified SoS will execute the
                        workflow in parallel mode and execute up to specified
                        processes concurrently. These include looped processes
                        within a step (with runtime option `concurrent=True`)
                        and steps with non-missing required files.
  -c CONFIG_FILE        A configuration file in the format of YAML/JSON. The
                        content of the configuration file will be available as
                        a dictionary CONF in the SoS script being executed.
  -t FILE [FILE ...]    One of more files or alias of other targets that will
                        be the target of execution. If specified, SoS will
                        execute only part of a workflow or multiple workflows
                        or auxiliary steps to generate specified targets.
  -b [BIN_DIR [BIN_DIR ...]]
                        Extra directories in which SoS will look for
                        executables before standard $PATH. This option
                        essentially prefix $PATH with these directories. Note
                        that the default value '~/.sos/bin' is by convention a
                        default directory for commands that are installed by
                        SoS. You can use option '-b' without value to disallow
                        commands under ~/.sos/bin.
  -q QUEUE              Task-processing queue. SoS by default uses a local
                        multiprocessing queue where tasks are executed by
                        different processes. Supported task queues include a
                        'rq' engine where tasks will be distributed to one or
                        more rq-workers with assistance from a redis server,
                        and a 'celery' quque where tasks will be distributed
                        to celery workers.
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information to standard output (default to
                        2).

Run mode options:
  SoS scripts are by default executed in run mode where all the script is
  run in dryrun mode to check syntax error, prepare mode to prepare
  resources, and run mode to execute the pipelines. Run mode options allow
  you to execute these steps selectively.

  -n                    Execute a workflow without executing any actions. This
                        can be used to check the syntax of a SoS file.
  -p                    Execute the workflow in preparation mode in which SoS
                        prepare the execution of workflow by, for example,
                        download required resources and docker images.
  -f                    Execute the workflow in a special run mode that
                        ignores saved runtime signatures and re-execute all
                        the steps.
  -F                    Execute the workflow in a special run mode that re-use
                        existing output files and recontruct runtime
                        signatures if output files exist.

Arbitrary parameters defined by the [parameters] step of the script, and
[parameters] steps of other scripts if nested workflows are defined in other
SoS files (option `source`). The name, default and type of the parameters are
specified in the script. Single value parameters should be passed using option
`--name value` and multi-value parameters should be passed using option
`--name value1 value2`.
```

Examples of the `sos run` include

```bash
sos run -h                       # get help message
sos run myscript                 # run default workflow defined in myscript
sos run myscript align           # run workflow align defined in myscript
sos run myscript align+call      # run a combined workflow align+call
sos run myscript align -h        # help message for the align workflow defined in myscript
sos run -n myscript align        # run align workflow in inspect mode 
```

## subcommand `dryrun`

This command execute the script in dryrun mode. It is alias to command `sos run -n`.

## subcommand `prepare`

This command execute the script in prepare mode. It is alias to command `sos run -p`.

## subcommand `convert`

```bash
$ sos convert -h
usage: sos convert [-h] [--html [FILENAME]] [--markdown [FILENAME]] [--term]
                   [--notebook [FILENAME]] [--sos [SCRIPT]] [-v {0,1,2,3,4}]
                   FILENAME [WORKFLOW]

The show command displays details of all workflows defined in a script,
including description of script, workflow, steps, and command line parameters.
The output can be limited to a specified workflow (which can be a subworkflow
or a combined workflow) if a workflow is specified.

positional arguments:
  FILENAME              File to be converted, can be a SoS script or a Jupyter
                        notebook.
  WORKFLOW              Workflow to be converted if the file being converted
                        is a SoS script.

optional arguments:
  -h, --help            show this help message and exit
  --html [FILENAME]     Generate a syntax-highlighted HTML file, write it to a
                        specified file, or view in a browser if no filename is
                        specified. Additional argument --raw can be used to
                        specify a URL to raw file, arguments --linenos and
                        --style can be used to customize style of html output.
                        You can pass an arbitrary name to option --style get a
                        list of available styles.
  --markdown [FILENAME]
                        Convert script or workflow to markdown format and
                        write it to specified file, or standard output if not
                        filename is specified.
  --term                Output syntax-highlighted script or workflow to the
                        terminal. Additional arguments --bg=light|dark
                        --lineno can be used to customized output.
  --notebook [FILENAME]
                        Convert script or workflow to jupyter notebook format
                        and write it to specified file, or standard output if
                        no filename is specified. If the input file is a
                        notebook, it will be converted to .sos (see option
                        --sos) then to notebook, resetting indexes and
                        removing all output cells.
  --sos [SCRIPT]        Convert specified Jupyter notebook to SoS format. The
                        output is the same as you use File -> Download as ->
                        SoS (.sos) from Jupyter with nbconvert version 4.2.0
                        or higher although you can customize output using
                        options --reorder (rearrange notebook cells with
                        execution order), --reset-index (reset indexes to 1,
                        2, 3, ..), --add-header (add section header [index] if
                        the cell does not start with a header), --no-index
                        (does not save cell index), --remove-magic (remove
                        cell magic), and --md-to-report (convert markdown cell
                        to code cell with report.)
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information to standard output (default to
                        2).

Extra command line argument could be specified to customize the style of html,
markdown, and terminal output.
```

The `--html` option is very useful in displaying SoS scripts with proper syntax highlighting for different embedded languages. It can be used to generate a nice-looking .html file if you run

```
sos convert myscript.sos --html myscript.sos.html
```

or generate a temporary file and display it in a browser if you do not specify an output filename.

```
sos convert myscript.sos --html myscript.sos.html
```

For example, the example in our [QuickStart Guide](https://github.com/bpeng2000/SOS/wiki/Quick-Start) can be displayed as

 [[/media/QuickStart.sos.html.png]]. 

It is also worth noting that you are a VIM user, it is highly recommended that you install the SoS syntax file so that your SoS scripts can be properly highlighted during editing. This is not only visually appealing but is very helpful in identifying syntax errors as you write. The same script in vim with a dark background would look look like 

 [[/media/QuickStart.sos.vim.png]]

## subcommand `config`

```
usage: sos config [-h] [-g] [-c CONFIG_FILE]
                  (--get [OPTION [OPTION ...]] | --unset OPTION [OPTION ...] | --set KEY VALUE [KEY VALUE ...])
                  [-v {0,1,2,3,4}]

The config command displays, set, and unset configuration variables defined in
global or local configuration files.

optional arguments:
  -h, --help            show this help message and exit
  -g, --global          If set, change global (~/.sos/config.yaml) instead of
                        local (.sos/config.yaml) configuration
  -c CONFIG_FILE, --config CONFIG_FILE
                        User specified configuration file in YAML format. This
                        file will not be automatically loaded by SoS but can
                        be specified using option `-c`
  --get [OPTION [OPTION ...]]
                        Display values of specified configuration. The
                        arguments of this option can be a single configuration
                        option or a list of option. Wildcard characters are
                        allowed to match more options (e.g. '*timeout',
                        quotation is needed to avoid shell expansion). If no
                        option is given, all options will be outputted.
  --unset OPTION [OPTION ...]
                        Unset (remove) settings for specified options. The
                        arguments of this option can be a single configuration
                        option or a list of option. Wildcard characters are
                        allowed to match more options (e.g. '*timeout', or '*'
                        for all options, quotation is needed to avoid shell
                        expansion).
  --set KEY VALUE [KEY VALUE ...]
                        --set KEY VALUE sets VALUE to variable KEY. The value
                        can be any valid python expression (e.g. 5 for integer
                        5 and '{"c": 2, "d": 1}' for a dictionary) with
                        invalid expression (e.g. val without quote) considered
                        as string. Syntax 'A.B=v' can be used to add {'B': v}
                        to dictionary 'A', and --set KEY VALUE1 VALUE2 ...
                        will create a list with multiple values.
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information to standard output (default to
                        2).

```

This subcommand can read/write three configuration files

1. **`~/.sos/config.yaml`** A global SoS configuration file. (option `--global`)
2. **`.sos/config.yaml`** Project specific SoS configuration file. (default)
3. **User specified** Any configuration file specified with option `-c` (`--config`).

The global and local configuration files will be loaded by default. Other configuration files can be specified using option `-c` of the `sos run` command. Options defined in these configuration file will be available to the script as dictionary `CONFIG`. 

Note that you can set complex datatypes from command line using Python expressions,

```
sos config --set a '{"b":1}'
```

although 

```
sos config --set a.b 1
```

is easier to write if you only need to set one key `b` in dictionary `a`.

## subcommand `remove`

```
$ sos remove -h
usage: sos remove [-h] [-t | -u] [-n] [-y] [FILE_OR_DIR [FILE_OR_DIR ...]]

Remove specified files and directories and their signatures (if available).
Optionally, you can remove only tracked files (input, output and intermediate
files of executed workflows) or untracked file from specified files and/or
directories.

positional arguments:
  FILE_OR_DIR  Files and directories to be removed, which should be under the
               current directory (default). All, tracked, or untracked files
               will be removed depending on other options ('-t' or '-u'). For
               safety reasons, files under the current directory have to be
               listed (not as files under .) to be removed.

optional arguments:
  -h, --help   show this help message and exit
  -t           Remove tracked files and their signatures from specified files
               and directories.
  -u           Remove untracked files from specified files and directories.
  -n           List files or directories to be removed, without actually
               removing them.
  -y           Remove files without confirmation, suitable for batch removal
               of files.
```

## subcommand `pack`

```
$ sos pack -h
usage: sos pack [-h] [-o OUTPUT] [-i [INCLUDE [INCLUDE ...]]]
                [-e [EXCLUDE [EXCLUDE ...]]] [-a] [-m MESSAGE] [-y]
                [-v {0,1,2,3,4}]
                [session]

positional arguments:
  session               ID of the session to be saved, which can be any number
                        of digits as long as it can uniquely determine a
                        workflow session. This parameter can be ignored if
                        only one session is available.

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        Output file, which can be a file with extension ".sar"
                        (the extension will be be automatically appended if
                        needed), or "-" for standard output (default).
  -i [INCLUDE [INCLUDE ...]], --include [INCLUDE [INCLUDE ...]]
                        Additional files or directories to be incldued in the
                        archive. SoS will archive all files under specified
                        directories, including hidden directories such as
                        ".git". Option --exclude could be used to exclude
                        these files.
  -e [EXCLUDE [EXCLUDE ...]], --exclude [EXCLUDE [EXCLUDE ...]]
                        Files that should be excluded from archive. The
                        parameter should be one or more patterns that match
                        the whole path (e.g. "output/*.log" or file or
                        directory names such as "tmp" or "*.bam".
  -a, --all             Include all tracked files even if they reside outside
                        of the current working directory.
  -m MESSAGE, --message MESSAGE
                        A short message to be included into the archive.
                        Because the message would be lost during unpacking, it
                        is highly recommended that you create a README file
                        and include it with option --include.
  -y, --yes             Overwrite output file if it already exists
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information to standard output (default to
                        2).
```

## subcommand `unpack`

```
$ sos unpack -h
usage: sos unpack [-h] [-d DEST] [-l] [-e] [-n] [-y] [-v {0,1,2,3,4}]
                  archive [files [files ...]]

positional arguments:
  archive               SoS archive saved by command sos pack
  files                 An optional list of files to be processed, which can
                        be exact filenames or patterns (e.g. "*.bam")

optional arguments:
  -h, --help            show this help message and exit
  -d DEST, --dest DEST  Directory where a sos archive would be unpacked.
                        Default to current directory.
  -l, --list            List content of the archive instead of extracting it.
                        The names, uncompressed file sizes and modification
                        dates and times of the specified files are printed,
                        along with totals for all files specified.
  -e, --external        Extract files outside of the project to their external
                        destinations. This option can be dangerous because it
                        can overwrite system files silently if accompanied
                        with option -y.
  -n, --no              Do not overwrite existing files without promoting
                        users.
  -y, --yes             Overwrite existing files without promoting users. This
                        option can be dangerous to use. Note that SoS checks
                        file signature and ignores existing files that are
                        identical to those in the archive.
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information to standard output (default to
                        2).
```

## Command `sos-runner`

Command `sos-runner` is a shortcut for ``sos run`` so

```bash
sos-runner script
```

is equivalent to

```bash
sos run script
```

This allows a SoS script to be executed directly if it is executable with shebang line

```
#!/usr/bin/env sos-runner
```